In [1]:
import dash
import dash_table
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import folium
from folium import plugins
from datetime import datetime
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

In [2]:
# df_nhsr_location = pd.DataFrame()
# df_nhsr = pd.DataFrame()
# carona_data_cs = pd.DataFrame()
# carona_data_cum = pd.DataFrame()
# res = pd.DataFrame()
# cases_forecast = pd.DataFrame()
# deaths_forecast_uk = pd.DataFrame()


# df_population_with_carona_case = pd.DataFrame()
# carona_data = pd.DataFrame() 
# df1  = pd.DataFrame()
# carona_data_cases_states = pd.DataFrame()
# cor_CACD = pd.DataFrame()
# df_ageGroup = pd.DataFrame()
# deaths_forecast = pd.DataFrame()
# conf_forecast = pd.DataFrame()
# rec_forecast = pd.DataFrame()

In [3]:
#load India data
df_ageGroup = pd.read_csv('AgeGroupDetails.csv')
df_cases = pd.read_csv('covid_19_india.csv')
df_hosp_beds = pd.read_csv('HospitalBedsIndia.csv')
df_testing = pd.read_csv('ICMRTestingDetails.csv')
df_Individual = pd.read_csv('IndividualDetails.csv')
df_population_of_india = pd.read_csv('population_india_census2011.csv')
df_with_utm = pd.read_csv("UTM ZONES of INDIA.csv")
df_cases.head()


df_cases["Active cases"] = df_cases.Confirmed - (df_cases.Cured + df_cases.Deaths)
df_cases.tail()

df_cases['Date'] =pd.to_datetime(df_cases.Date,dayfirst=True)
#Daily Cases in India Datewise
carona_data = df_cases.groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Confirmed',ascending = True)
carona_data['Daily Cases'] = carona_data['Confirmed'].sub(carona_data['Confirmed'].shift())
carona_data['Daily Cases'].iloc[0] = carona_data['Confirmed'].iloc[0]
carona_data['Daily Cases'].iloc[0] = carona_data['Daily Cases'].astype(int).iloc[0]
carona_data.tail()


#Total Cases in India State Datewise
carona_data_cases_states = df_cases.groupby(['Date','State/UnionTerritory','Confirmed'])['Cured','Deaths','Active cases'].sum().reset_index().sort_values('Confirmed',ascending = False)
carona_data_cases_states.head()


df1= df_cases[df_cases['Date']=='04/04/20'].sort_values('Confirmed',ascending = False)
df1.head()

#Statewise Total Cases
df_new = df1.groupby(['State/UnionTerritory','Cured','Deaths','Active cases'])['Confirmed'].sum().reset_index().sort_values('Confirmed',ascending = False)
df_new.head()

#Using Merge two join two diffrent data frames and then sorting them in ascending
df_population_with_carona_case = df_population_of_india.merge(df_new, left_on='State / Union Territory', right_on='State/UnionTerritory')
df_population_with_carona_case=df_population_with_carona_case.drop(labels=['State/UnionTerritory','Sno'],axis=1)
df_population_with_carona_case=df_population_with_carona_case.sort_values('Confirmed',ascending=False)
df_population_with_carona_case=df_population_with_carona_case.drop(['Population','Rural population','Urban population','Area','Density','Gender Ratio'],axis=1)
df_population_with_carona_case.head()


df_pop_caro_utm = df_population_with_carona_case.merge(df_with_utm , left_on='State / Union Territory', right_on='State / Union Territory')
df_pop_caro_utm.head()


#India's Map with Statewise data of Total Cases,Deaths and Cure
India_map = folium.Map(location=[20.5937, 78.9629],zoom_start=4.55)
fg=folium.FeatureGroup(name="my map")
fg.add_child(folium.GeoJson(data=(open('states_of_india.json','r',encoding='utf-8-sig').read())))
India_map.add_child(fg)
for lat,lan,name,cured,deaths,cases in zip(df_pop_caro_utm['Latitude'],df_pop_caro_utm['Longitude'],df_pop_caro_utm['State / Union Territory'],df_pop_caro_utm['Cured'],df_pop_caro_utm['Deaths'],df_pop_caro_utm['Confirmed']):
    if(deaths == 0):
        folium.Marker(location=[lat,lan],popup="<b>State  : </b>"+name+ "<br> <b>Total Cases : </b> "+str(cases)+"<br> <b>Deaths : </b> "+str(deaths)+"<br> <b>Cured : </b> "+str(cured)).add_to(India_map)
    else:
        folium.Marker(location=[lat,lan],popup="<b>State  : </b>"+name+ "<br> <b>Total Cases : </b> "+str(cases)+"<br> <b>Deaths : </b> "+str(deaths)+"<br> <b>Cured : </b> "+str(cured),icon=folium.Icon(color="red")).add_to(India_map)
India_map.save('india_map.html')

#Total Cases,Active Cases,Cured,Deaths from Corona Virus in India
cor_CACD = df_cases.groupby(['Date'])['Confirmed','Active cases','Cured','Deaths'].sum().reset_index().sort_values('Date',ascending=False)
cor_CACD.head()

#Forecasting total number of cases WorldWide
from fbprophet import Prophet
confirmed = df_cases.groupby("Date").sum()["Confirmed"].reset_index()
deaths = df_cases.groupby("Date").sum()["Deaths"].reset_index()
recovered = df_cases.groupby("Date").sum()["Cured"].reset_index()
recovered.tail()

deaths.columns = ['ds','y']
deaths['ds'] = pd.to_datetime(deaths['ds'])
deaths.tail()
m=Prophet(interval_width = 0.95, daily_seasonality=True,yearly_seasonality=True)
m.fit(deaths)
future=m.make_future_dataframe(periods=3)
#predicting the future witn date, upper and lower limit of y value
deaths_forecast = m.predict(future)
deaths_forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(7)

confirmed.columns = ['ds','y']
confirmed['ds'] = pd.to_datetime(confirmed['ds'])
confirmed.tail()
m=Prophet(interval_width = 0.95, daily_seasonality=True,yearly_seasonality=True)
m.fit(confirmed)
future=m.make_future_dataframe(periods=2)
#predicting the future witn date, upper and lower limit of y value
conf_forecast = m.predict(future)
conf_forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(7)

recovered.columns = ['ds','y']
recovered['ds'] = pd.to_datetime(recovered['ds'])
recovered.tail()
m=Prophet(interval_width = 0.95, daily_seasonality=True,yearly_seasonality=True)
m.fit(recovered)
future=m.make_future_dataframe(periods=2)
#predicting the future witn date, upper and lower limit of y value
rec_forecast = m.predict(future)
rec_forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(2)


C:\Users\itzai\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\itzai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\itzai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:61: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,ds,yhat,yhat_lower,yhat_upper
66,2020-04-05,189.762784,184.978117,194.448170
67,2020-04-06,190.062970,185.797020,194.512148


In [4]:
#load uk data
df_nhsr = pd.read_csv('../UK/NHSR_Cases_table.csv')
df_cases_UK = pd.read_csv('../UK/CountyUAs_cases_table.csv')
df_daily_conf = pd.read_excel('../UK/DailyConfirmedCases.xlsx')
df_tot_cases = pd.read_csv('../UK/full-list-total-tests-for-covid-19.csv')
df_casespermillion = pd.read_csv('../UK/full-list-cumulative-total-tests-per-million.csv')

df_nhsr.head()
df_nhsr_location = df_nhsr[['NHSRNm','TotalCases']].sort_values(ascending=False, by = 'TotalCases')
df_nhsr_location.head()

df_nhsr["TotalCases"] = df_nhsr["TotalCases"].str.replace(',', '').astype(int)

#Total Cases,Active Cases,Cured,Deaths from Corona Virus in India
carona_data_cs = df_daily_conf.groupby(['DateVal'])[['CMODateCount','DailyDeaths']].sum().reset_index().sort_values('DateVal',ascending=False)
carona_data_cs.head()

carona_data_cum = df_daily_conf.groupby(['DateVal'])[['CumCases','CumDeaths']].sum().reset_index().sort_values('DateVal',ascending=False)
carona_data_cum.head()

df_tot_cases['Date'] = pd.to_datetime(df_tot_cases['Date'])
df_tot_cases = df_tot_cases[df_tot_cases['Entity'] == 'United Kingdom - people tested']
df_tot_cases['dd'] = df_tot_cases['Cumulative total tests']-df_tot_cases['Cumulative total tests'].shift()
df_daily_conf = df_daily_conf[['DateVal','CumCases','CumDeaths']]
res = pd.merge(df_tot_cases[:-1], df_daily_conf, left_on='Date', right_on='DateVal',how = 'outer')
res = res[['Date','Cumulative total tests','CumCases','CumDeaths']].fillna(0)
res.tail(5)


#Forecasting total number of cases WorldWide
from fbprophet import Prophet

df_daily_conf.head(5)
cum_Cases = df_daily_conf[['DateVal','CumCases']]
cum_Deaths = df_daily_conf[['DateVal','CumDeaths']].fillna(0)

cum_Cases.columns = ['ds','y']
cum_Cases['ds'] = pd.to_datetime(cum_Cases['ds'])
m=Prophet(interval_width = 0.95, daily_seasonality=True,yearly_seasonality=True)
m.fit(cum_Cases)
future=m.make_future_dataframe(periods=7)
#predicting the future witn date, upper and lower limit of y value
cases_forecast = m.predict(future)
cases_forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(7)

cum_Deaths.columns = ['ds','y']
cum_Deaths['ds'] = pd.to_datetime(cum_Deaths['ds'])
m=Prophet(interval_width = 0.95, daily_seasonality=True,yearly_seasonality=True)
m.fit(cum_Deaths)
future=m.make_future_dataframe(periods=7)
#predicting the future witn date, upper and lower limit of y value
deaths_forecast = m.predict(future)
deaths_forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(7)

C:\Users\itzai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,yhat,yhat_lower,yhat_upper
64,2020-04-04,4504.404051,4469.252424,4540.578109
65,2020-04-05,5561.254312,5525.045112,5595.170527
66,2020-04-06,6817.759933,6783.846704,6852.728380
67,2020-04-07,8326.808265,8292.845964,8360.981491
68,2020-04-08,10102.441861,10067.943027,10138.086703
69,2020-04-09,12164.509843,12129.111063,12198.853428
70,2020-04-10,14542.377277,14508.790027,14579.251912


In [5]:
tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

app.layout = html.Div([
    dcc.Tabs(id="tabs-styled-with-inline", value='t1', children=[
        dcc.Tab(label='India', value='t1', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='UK', value='t2', style=tab_style, selected_style=tab_selected_style),
    ], style=tabs_styles),
    html.Div(id='tabs-content-inline')
])

In [6]:
def uk_data():
        
        return html.Div([

        html.H1(children = 'UK Data',
                style = {
                    'textAlign':'center',
                    'color':'#ff0000'
                }
                ),
        dash_table.DataTable(
                    id='table1',
                    columns=[{"name": i, "id": i} for i in df_nhsr_location.columns],
#                     filter_action="native",
#                     sort_action="native",
#                     sort_mode="multi",
                    data = df_nhsr_location.to_dict("rows"),
                    style_header={'backgroundColor': '#AFEEEE',
                                  'height': '30px',
                                  'border': '1px solid black',
                                  'font-size': '14px',
                                 'fontWeight': 'bold',
                                 'text-align': 'center'},
                    style_table={
                                'maxHeight': '500px',
                                'maxWidth': '300px',
                                'overflowX': 'scroll',
                                'overflowY': 'scroll',
                                'border': 'thin lightgrey solid'
                            },
                    style_cell={
                                'height': 'auto',
                                'backgroundColor': 'rgb(255,255,255)',
                                'border': '1px solid blue' ,
                                'maxWidth': '150px'
                    },
                    style_data_conditional=[
                                        {
                                            'if': {'row_index': 'odd'},
                                            'backgroundColor': 'rgb(192,192,192)'
                                        }
                                ]    
        ),



        html.Div(dcc.Graph(
            id = 'Scatter Chart 4',
            figure = {
                'data':[
                    (
                        go.Pie(labels=df_nhsr['NHSRNm'], 
                               values=df_nhsr['TotalCases'])
                    )
                ],
                'layout':go.Layout(
                    title= 'Till Date Location wise Total Cases in UK'
                )
            }
        ), style={'display': 'inline-block'}),


        html.Div(dcc.Graph(
            id = 'Scatter Chart 5',
            figure = {
                'data':[
                        go.Scatter(x=carona_data_cs['DateVal'], y=carona_data_cs['CMODateCount'],mode='lines+markers',name='CMODateCount'),
                        go.Scatter(x=carona_data_cs['DateVal'], y=carona_data_cs['DailyDeaths'], mode='lines+markers',name='DailyDeaths'),
                ],
                'layout':go.Layout(
                     title= 'Curve Showing Different Cases from COVID-19 in UK',
                     xaxis = {'title':'Date'},
                     yaxis = {'title':'Daily Cases'}
                )

            }
        ), style={'display': 'inline-block'}
                ),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 6',
            figure = {
                'data':[
                        go.Scatter(x=carona_data_cum['DateVal'],y=carona_data_cum['CumCases'],mode='lines+markers',name='CMODateCount'),
                        go.Scatter(x=carona_data_cum['DateVal'],y=carona_data_cum['CumDeaths'],mode='lines+markers',name='CMODateCount'),
                ],
                'layout':go.Layout(
                     title = 'Curve Showing Commulative Cases from COVID-19 in UK',
                     yaxis = {'title':"Total cases"},
                     xaxis = {'title':"Date"},
                )
            }
        ), style={'display': 'inline-block'}
                ),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 7',
            figure = {
                'data':[
                        go.Scatter(x=res['Date'], y=res['Cumulative total tests'],mode='lines+markers',name='Cumulative total tests'),
                        go.Scatter(x=res['Date'], y=res['CumCases'], mode='lines+markers',name='CumCases'),
                        go.Scatter(x=res['Date'], y=res['CumDeaths'],mode='lines+markers',name='CumDeaths'),
                ],
                'layout':go.Layout(
                     title= 'Curve Showing comparison Tests vs Cases vs Deaths from COVID-19 in UK',
                     xaxis = {'title':'Date'},
                     yaxis = {'title':'Daily Cases'}
                )

            }
        ), style={'display': 'inline-block'}
                ),
            
        html.Div(dcc.Graph(
            id = 'Scatter Chart 7',
            figure = {
                'data':[
                        go.Scatter(x=cases_forecast['ds'], y=cases_forecast['yhat'].astype(int),mode='lines+markers',name='Deaths'),
                        go.Scatter(x=deaths_forecast['ds'], y=deaths_forecast['yhat'].astype(int), mode='lines+markers',name='Confirmed'),
                ],
                'layout':go.Layout(
                     title= 'Curve Showing Next Week Predicted Cases from COVID-19 in UK',
                     xaxis = {'title':'Date'},
                     yaxis = {'title':'Daily Cases'}
                )

            }
        )
               ),


    ]#, style={'display': 'one'}
        )


In [7]:
def india_data():
        return html.Div([

        html.H1(children = 'India Data',
                style = {
                    'textAlign':'center',
                    'color':'#ff0000'
                }
               ),
        dash_table.DataTable(
                    id='table1',
                    columns=[{"name": i, "id": i} for i in df_population_with_carona_case.columns],
#                     filter_action="native",
#                     sort_action="native",
#                     sort_mode="multi",
                    data = df_population_with_carona_case.to_dict("rows"),
                    style_header={'backgroundColor': '#AFEEEE',
                                  'height': '40px',
                                  'border': '1px solid black',
                                  'font-size': '14px',
                                 'fontWeight': 'bold'},
                    style_table={
                                'maxHeight': '500px',
                                'maxWidth': '500px',
                                'overflowX': 'scroll',
                                'overflowY': 'scroll',
                                'border': 'thin lightgrey solid'
                            },
                    style_cell={
                                'height': 'auto',
                                'backgroundColor': 'rgb(255,255,255)',
                                'border': '1px solid blue' ,
                                'maxWidth': '180px'
                    },
                    style_data_conditional=[
                                        {
                                            'if': {'row_index': 'odd'},
                                            'backgroundColor': 'rgb(192,192,192)'
                                        }
                                ]    
        ),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 1',
            figure = {
                'data':[
                    (
                        go.Bar(x=carona_data['Date'], 
                            y=carona_data['Daily Cases'],
                            marker=dict(color='#00FF00'),
                            #marker=dict(color=carona_data['Daily Cases'])
                           )
                    )
                ],
                'layout':go.Layout(
                    title= 'Daily Cases in India Datewise',
                    xaxis = {'title':'Date'},
                    yaxis = {'title':'Daily Cases'}
                )
            }
        ), style={'display': 'inline-block'}),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 2',
            figure = {
                'data':[
                    (
                        go.Bar(x=df1['State/UnionTerritory'], 
                            y=df1['Confirmed'],
                               #marker=dict(color='#00FF00'),
                            marker=dict(color=df1['Confirmed'])
                           )
                    )
                ],
                'layout':go.Layout(
                    title= 'Till Date Satewise Total Cases in India',
                    xaxis = {'title':'State/UnionTerritory'},
                    yaxis = {'title':'Confirmed Cases'}
                )
            }
        ), style={'display': 'inline-block'}
                ),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 3',
            figure = {
                'data':[
                    (
                        go.Bar(x=carona_data_cases_states['Date'], 
                            y=carona_data_cases_states['Confirmed'],
                            marker=dict(color=carona_data_cases_states['Confirmed'])#,
                               #hovertemplate = carona_data_cases_states[['State/UnionTerritory','Active cases','Deaths','Cured']]
                           )
                    )
                ],
                'layout':go.Layout(
                    title= 'Till Date Satewise Total Cases in India',
                    xaxis = {'title':'State/UnionTerritory'},
                    yaxis = {'title':'Confirmed Cases'},
                    barmode='group'
                )
            }
        ), style={'display': 'inline-block'}),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 4',
            figure = {
                'data':[
                    (
                        go.Pie(labels=df_population_with_carona_case['State / Union Territory'], 
                               values=df_population_with_carona_case['Confirmed'])
                    )
                ],
                'layout':go.Layout(
                    title= 'Till Date Satewise Total Cases in India',
                    xaxis = {'title':'State/UnionTerritory'},
                    yaxis = {'title':'Confirmed Cases'}
                )
            }
        ), style={'display': 'inline-block'}),

        html.Div(
            html.Iframe(id='Map', 
                        srcDoc=open('india_map.html', 'r').read(), \
                        width='100%', 
                        height='600'
                       )
        ),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 5',
            figure = {
                'data':[
                        go.Scatter(x=cor_CACD['Date'], y=cor_CACD['Confirmed'],mode='lines+markers',name='Confirmed'),
                        go.Scatter(x=cor_CACD['Date'], y=cor_CACD['Active cases'], mode='lines+markers',name='Active cases'),
                        go.Scatter(x=cor_CACD['Date'], y=cor_CACD['Cured'], mode='lines+markers',name='Cured'),
                        go.Scatter(x=cor_CACD['Date'], y=cor_CACD['Deaths'], mode='lines+markers',name='Deaths')
                ],
                'layout':go.Layout(
                     title= 'Curve Showing Different Cases from COVID-19 in India',
                     xaxis = {'title':'Date'},
                     yaxis = {'title':'Daily Cases'}
                )

            }
        ), style={'display': 'inline-block'}
                ),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 6',
            figure = {
                'data':[
                        go.Scatter(x=df_ageGroup['AgeGroup'],y=df_ageGroup['TotalCases'],
                                   line_shape='spline', fill='tonexty',fillcolor = 'orange')
                ],
                'layout':go.Layout(
                     title = '"Age wise Trend"',
                     yaxis = {'title':"Total cases"},
                     xaxis = {'title':"Age Group"},
                    plot_bgcolor='rgb(275, 270, 273)',
                    width=800,
                    height=600
                )
            }
        ), style={'display': 'inline-block'}
                ),

        html.Div(dcc.Graph(
            id = 'Scatter Chart 7',
            figure = {
                'data':[
                        go.Scatter(x=deaths_forecast['ds'], y=deaths_forecast['yhat'].astype(int),mode='lines+markers',name='Deaths'),
                        go.Scatter(x=conf_forecast['ds'], y=conf_forecast['yhat'].astype(int), mode='lines+markers',name='Confirmed'),
                        go.Scatter(x=rec_forecast['ds'], y=rec_forecast['yhat'].astype(int), mode='lines+markers',name='Recovered'),
                ],
                'layout':go.Layout(
                     title= 'Curve Showing Next Week Predicted Cases from COVID-19 in India',
                     xaxis = {'title':'Date'},
                     yaxis = {'title':'Daily Cases'}
                )

            }
        )
                ),


    ]#, style={'display': 'none'}
        )


In [8]:
@app.callback(Output('tabs-content-inline', 'children'),
              [Input('tabs-styled-with-inline', 'value')])
def render_content(tab):
    if tab == 't1':
        app.layout = html.Div([html.H1()])
        return india_data()

    elif tab == 't2':
        app.layout = html.Div([html.H1()])
        return uk_data()

In [ ]:
if __name__ == '__main__':
    app.run_server(port = 4050)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:4050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [11/Apr/2020 21:55:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2020 21:55:39] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2020 21:55:39] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2020 21:55:39] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2020 21:55:39] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2020 21:55:39] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Apr/2020 21:55:39] "GET /_dash-compo